# <ins> Finalizing the boundary-setting </ins>
Based on the last three models, we managed to obtain a pretty solid mapping of the areas of Eindhoven which could properly fit the definitions provided by Fieke.
It is crucial now to define these boundaries as accurately as possible; the goal of this notebook is thus to see how we can overlay Alex and Sjoerd's findings to the current mappings, and extract a more detailed map with clearly defined boundaries.


## Overlaying the maps
First, as I mentioned, we need to overlay the final mapping of Eindhoven that I designed to the mapping provided by Alex and Sjoerd; this will give us the most accurate mapping of the zones we will be working with, and will allow us to properly model the points of connection we want to place later on.

Analyzing the map the Built Environment guys provided, you can see that **all of the regions fit inside the boundary-passing model zones!** This means that what we need to do at this point is to merge and fit the specific OSM boundaries we will use to these distinctions, and create a workable map where all of the regions are split in this manner. The most accurate manner of doing this would be to manually extract the polygons using GeoJSON or by using QGIS and referencing the two maps; I opted to use the first approach, since we then can easily re-shape the neighborhoods if new data is provided.

You can see the file containing these boundaries **refinedMap.geojson** in the same directory as this Python notebook; after loading using folium and geopandas, it looks as follows:




In [29]:
import geopandas as gpd
import osmnx as ox
import json
import folium
from IPython.display import display, HTML

with open("refinedMap.geojson") as openedFile:
    polygonGeoJSON = json.load(openedFile)

# Convert GeoJSON to GeoDataFrame
geoDataEindhoven = gpd.GeoDataFrame.from_features(polygonGeoJSON["features"])
polygon = geoDataEindhoven.geometry.values[0]

# Create a Folium map centered on Eindhoven
eindhoven_map = folium.Map(location=[51.4416, 5.4697], zoom_start=12, tiles="cartodbpositron")

# Add the GeoJSON data to the map
folium.GeoJson(
    polygonGeoJSON,
    name="Eindhoven Boundaries",
    style_function=lambda feature: {
        "fillColor": "blue",
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.5,
    },
).add_to(eindhoven_map)
display(HTML(eindhoven_map._repr_html_()))

Now, we convert the GeoJSON data that I plotted into a working polygon, so that we may use it with OSMNx for optimal point plotting.

In [20]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import json

with open("refinedMap.geojson") as openedFile:
    polygonGeoJSON = json.load(openedFile)
    
geoDataEindhoven = gpd.GeoDataFrame.from_features(polygonGeoJSON["features"])
polygon = geoDataEindhoven.geometry.values[0]


Everything's converted - we can move on to finding the points we will use for our hubs.

## Finding low-efficiency zones

I am still brainstorming if there is a better way of going about this, but for now, the best approach would be to do what we presented in the midterm presentation: highlighting low-scoring zones in terms of 15-minute walkability, and then further refining where within these zones it would be best to place the bikes.

In [ ]:
from shapely.geometry import Point, Polygon
import numpy as npo
import folium

with open("refinedMap.geojson") as openedFile:
    polygonGeoJSON = json.load(openedFile)

geoDataEindhoven = gpd.GeoDataFrame.from_features(polygonGeoJSON["features"])
polygons = geoDataEindhoven.geometry

# I use the same tags I used before, since we agreed on them as a form of baseline, and the map Sjoerd showed used very
# similar tags. I added a few things to the "leisure" and "shop" tags because I realized that they are not as varied
# as the "office" and "amenity" tags. 
tags = {
    'office': ['yes'],  # All workplaces, since OSM has an incredible amount of identifiers.
    'shop': ['supermarket', 'convenience', 'bakery', 'greengrocer', 'butcher'],
    'amenity': [
        'pharmacy', 'hospital', 'clinic', 'doctors', 
        'school', 'kindergarten', 'college', 'university',  
        'cafe', 'restaurant', 'bar', 'cinema', 'theatre',  
        'community_centre', 'library', 'bicycle_rental',
        'place_of_worship'
        # Place of worship can be considered "key" ammenity for some, so should be included.
    ],
    'leisure': [
        'park', 'fitness_centre', 'sports_centre', 'stadium', 
        'dog_park', 'pitch', 'swimming_pool', 'playground',
        'nature_reserve'
    ],
    'shop' : [
        'supermarket', 'convenience', 'bakery', 'greengrocer', 'butcher',
        'department_store', 'general', 'cosmetics', 'stationery'
    ]
}

# First, we generate a grid of poinst so that this may actually be treated as a graphing problem.
def generateGrid(polygon, spacing=0.0015):
    minx, miny, maxx, maxy = polygon.bounds
    x_coords = npo.arange(minx, maxx, spacing)
    y_coords = npo.arange(miny, maxy, spacing)
    points = [Point(x,y) for x in x_coords for y in y_coords if polygon.contains(Point(x,y))]
    return points

# Then, we may simply analyze the polygons the same way we did way back in the original simulations,
# and give each polygon a score according to its accessibility.
lowScoreZones = []
for polygon in polygons:
    # As usual, needs some debugging messages
    print("Processing a new polygon... as always, might take a while.")
    
    # Generate a grid of points within the polygon - this is based on size of polygon, so 
    # there are obviously more points in larger polygons. Smallest ones has ~15, largest ~67
    grid_points = generateGrid(polygon)
    print(f"Generated {len(grid_points)} grid points within the polygon.")
    
    # Query OSM for the specified tags
    amenities = ox.features_from_polygon(polygon, tags)
    print(f"Found {len(amenities)} amenities in the polygon.")
    
    # Check accessibility for each grid point
    lowScoringPoints = []
    amenityCount = []
    
    for point in grid_points:
        # Create a 15-minute walking isochrone (approximately 1.2 km radius)
        isochrone = point.buffer(0.012)  # 1.2 km in degrees
        
        # Check if the isochrone contains any amenities
        accessible_amenities = amenities[amenities.intersects(isochrone)]
        amenityCount = len(accessible_amenities)
        amenityCounts.append(amenityCount)
    
    print(f"Identified {len(low_score_points)} low-scoring points in the polygon.")
    lowScoreZones.append(gpd.GeoDataFrame(geometry=low_score_points))
    
# Now, we need to visualize this! 
# We do what we did earlier, and convert to a single GeoDataFrame.
lowScoreZonesGDF = gpd.GeoDataFrame(pd.concat(lowScoreZones, ignore_index=True))
eindhoven_map = folium.Map(location=[51.4416, 5.4697], zoom_start=12, tiles="cartodbpositron")

# Obvious LLM-generated map is obvious. Bleeeh.

# Add the generated grid points to the map
for polygon in polygons:
    # Generate a grid of points within the polygon
    grid_points = generateGrid(polygon)
    
    # Add each grid point to the map
    for point in grid_points:
        folium.CircleMarker(
            location=[point.y, point.x],
            radius=2,  # Smaller radius for grid points
            color="green",
            fill=True,
            fill_opacity=0.6,
        ).add_to(eindhoven_map)

# Add the original polygons
folium.GeoJson(
    polygonGeoJSON,
    name="Eindhoven Boundaries",
    style_function=lambda feature: {
        "fillColor": "blue",
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.2,
    },
).add_to(eindhoven_map)

eindhoven_map

Processing a new polygon... as always, might take a while.
Generated 640 grid points within the polygon.
Found 358 amenities in the polygon.
Identified 0 low-scoring points in the polygon.
Processing a new polygon... as always, might take a while.
Generated 499 grid points within the polygon.
Found 571 amenities in the polygon.
Identified 0 low-scoring points in the polygon.
Processing a new polygon... as always, might take a while.
Generated 229 grid points within the polygon.
Found 103 amenities in the polygon.
Identified 0 low-scoring points in the polygon.
Processing a new polygon... as always, might take a while.
Generated 254 grid points within the polygon.
Found 172 amenities in the polygon.
Identified 0 low-scoring points in the polygon.
Processing a new polygon... as always, might take a while.
Generated 183 grid points within the polygon.
Found 67 amenities in the polygon.
Identified 0 low-scoring points in the polygon.
